In [2]:
from mangaki.utils.als import MangakiALS
from mangaki.utils.data import Dataset
from mangaki.utils.algo import fit_algo

#ratings = Rating.objects.values_list('user_id', 'work_id', 'choice')
#fit_algo('als', ratings)
algo = MangakiALS(20)
algo.load(algo.get_backup_filename())
dataset = Dataset()
dataset.load('ratings-' + algo.get_backup_filename())

In [15]:
class Tree:
    def __init__(self):
        pass

    def get_bulk(self, history = None):
        if history is None:
            return [13, 15, 131, 137, 8765]
        else:
            print('second run')
            return [2569, 2336, 3393, 3400, 1089]

In [16]:
import numpy as np

class NewUser:
    def __init__(self, model, my_user_id, truth):
        self.user_id = my_user_id
        self.model = model
        self.truth = truth
        self.history = []
        # clf = LinearRegression(fit_intercept=False)
        # clf2 = LogisticRegression(fit_intercept=False)
        """if len(set(yl)) >= 2:
            happened += 1
            clf2 = LogisticRegression(fit_intercept=False)
            # clf.fit(X, y)
            clf2.fit(X, yl)
            clf.coef_ = clf2.coef_
            clf.intercept_ = 0."""

    def ask(self, work_ids):
        # r_bonus
        # self.model = MangakiALS(NB_COMPONENTS)
        # self.model.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
        # self.model.fit(dataset.anonymized.X[r_train + r_bonus], dataset.anonymized.y[r_train + r_bonus])
        replied = []
        choices = []
        for work_id in work_ids:
            if work_id in self.truth:
                replied.append(work_id)
                choices.append(self.truth[work_id])
        self.history.append((work_ids, replied, choices))
        #print(replied, choices)
        print('Reply rate', len(choices) / len(work_ids))
        mean = np.mean(choices)
        self.model.means[self.user_id] = mean
        bonus_ratings = {self.user_id: {work_id: choice - mean for work_id, choice in zip(work_ids, choices)}}
        print(bonus_ratings)
        self.model.fit_user(self.user_id, bonus_ratings)
    
    def predict(self, work_ids):
        return self.model.predict(np.array([[self.user_id, work_id] for work_id in work_ids]))

    def get_truth(self, work_ids):
        return [self.truth.get(work_id) for work_id in work_ids]

    def compute_error(self, work_ids):
        y_pred = self.predict(work_ids)
        print(y_pred)
        y_true = self.get_truth(work_ids)
        print('was:', y_true)
        print('RMSE:', self.model.compute_rmse(y_pred, y_true))
    
    def get_history(self):
        flat_history = {}
        for work_ids, replied, choices in self.history:
            for work_id, choice in zip(replied, choices):
                flat_history[work_id] = choice
        return flat_history

In [22]:
def get_truth(my_user_id, dataset):
    truth = {}
    for (user_id, work_id), choice in zip(dataset.anonymized.X, dataset.anonymized.y):
        if user_id == my_user_id:
            truth[work_id] = choice
    return truth

truth = get_truth(1036, dataset)

In [23]:
JURY = [137, 1089, 2569, 2336, 3393, 3400, 3900, 242]

user = NewUser(algo, 1036, truth)
# ALS train
user.compute_error(JURY)
# Restart with multistage tree
tree = Tree()
work_ids = tree.get_bulk()
#work_ids = list(truth.keys())[:5]
print(work_ids)
user.ask(work_ids)
#user.compute_error(work_ids)
user.compute_error(JURY)
work_ids = tree.get_bulk(user.get_history())
print(work_ids)
user.ask(work_ids)
#user.compute_error(work_ids)
user.compute_error(JURY)

[ 0.48629922  0.48119145  0.10084038 -0.2811303   1.53073638  0.64427437
  0.57450067  1.05109368]
was: [None, None, 0.10000000000000001, None, 2.0, None, None, 2.0]
RMSE: 0.611182530697
[13, 15, 131, 137, 8765]
Reply rate 0.4
{1036: {13: -2.0, 15: 2.0}}
[ 0.13802336  0.13291559 -0.24743548 -0.62940616  1.18246052  0.29599851
  0.2262248   0.70281782]
was: [None, None, 0.10000000000000001, None, 2.0, None, None, 2.0]
RMSE: 0.907701095584
second run
[2569, 2336, 3393, 3400, 1089]
Reply rate 0.4
{1036: {2569: -0.95000000000000007, 2336: 0.94999999999999996}}
[ 1.18802336  1.18291559  0.80256452  0.42059384  2.23246052  1.34599851
  1.2762248   1.75281782]
was: [None, None, 0.10000000000000001, None, 2.0, None, None, 2.0]
RMSE: 0.450456739622


In [12]:
dataset.titles = {}
for work_id, title in Work.objects.values_list('id', 'title'):
    dataset.titles[work_id] = title

In [20]:
sorted([key, dataset.titles[dataset.decode_work[key]], truth[key]] for key in truth.keys())[:15]

[[13, 'Death Note', 2.0],
 [53, 'Berry Dynamite', 2.0],
 [74, 'Magi - The Labyrinth of Magic', 2.0],
 [242, 'Naruto', 0.10000000000000001],
 [364, 'Gourmet solitaire (le)', 2.0],
 [400, 'Tokyo ghoul', 2.0],
 [445, '20th century boys', 0.10000000000000001],
 [449, 'Lost Paradise', 0.10000000000000001],
 [517, 'Monster', 2.0],
 [571, 'Acid Town', 0.5],
 [588, 'Nausicaa', 2.0],
 [704, 'Bride Stories', 2.0],
 [716, "Vision d'Escaflowne", 0.10000000000000001],
 [717, 'Bamboo Blade', 0.5],
 [718, 'To Love', -2.0]]

In [98]:
dataset.anonymized.X

array([[ 852, 5546],
       [1596, 8593],
       [1036, 6178],
       ..., 
       [ 426,  401],
       [1139, 5611],
       [ 665, 9257]])